# Imports

In [ ]:
import pandas as pd

import cyclops.query.process as qp
from cyclops.processors.column_names import (
    DIAGNOSIS_CODE,
    ENCOUNTER_ID,
    LENGTH_OF_STAY_IN_ER,
)
from cyclops.query import gemini
from cyclops.query.gemini import get_interface

# Query

In [ ]:
def get_cohort() -> pd.DataFrame:
    """Get delirium cohort."""
    table = gemini.patient_encounters().query
    table = qp.DropNulls("del_present")(table)
    table = qp.ConditionEquals("del_present", 3, not_=True)(table)

    return get_interface(table).run()


def get_room_transfers_for_cohort(cohort: pd.DataFrame) -> pd.DataFrame:
    """Get room transfer info for cohort.

    Parameters
    ----------
    cohort: pandas.DataFrame
        Cohort data.

    Returns
    -------
    pandas.DataFrame
        Cohort with room transfer data added.

    """
    table = gemini.room_transfers().query
    table = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "room_transfers")}
    )(table)
    room_transfers = get_interface(table).run()
    cohort = cohort.merge(room_transfers, on=ENCOUNTER_ID, how="left")

    return cohort


def get_scu_transfers_for_cohort(cohort: pd.DataFrame) -> pd.DataFrame:
    """Get SCU transfer info for cohort.

    Parameters
    ----------
    cohort: pandas.DataFrame
        Cohort data.

    Returns
    -------
    pandas.DataFrame
        Cohort with SCU transfer data added.

    """
    table = gemini.get_table("ip_scu")
    table = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "icu_transfers")}
    )(table)
    ip_scu = get_interface(table).run()
    cohort = cohort.merge(ip_scu, on=ENCOUNTER_ID, how="left")
    cohort["icu_transfers"] = cohort["icu_transfers"].fillna(0)

    return cohort


def get_er_for_cohort(cohort: pd.DataFrame) -> pd.DataFrame:
    """Get ER info for cohort.

    Parameters
    ----------
    cohort: pandas.DataFrame
        Cohort data.

    Returns
    -------
    pandas.DataFrame
        Cohort with ER data added.

    """
    table = gemini.er_admin().query
    table = qp.FilterColumns(
        [
            ENCOUNTER_ID,
            "admit_via_ambulance",
            "triage_level",
            LENGTH_OF_STAY_IN_ER,
        ]
    )(table)
    er_info = get_interface(table).run()
    cohort = cohort.merge(er_info, on=ENCOUNTER_ID, how="left")

    return cohort


def get_bt_for_cohort(cohort: pd.DataFrame) -> pd.DataFrame:
    """Get blood transfusions info for cohort.

    Parameters
    ----------
    cohort: pandas.DataFrame
        Cohort data.

    Returns
    -------
    pandas.DataFrame
        Cohort with blood transfusions data added.

    """
    table = gemini.blood_transfusions().query
    total = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "total_transfusions")}
    )(table)
    unique = qp.GroupByAggregate(
        [ENCOUNTER_ID, "blood_product_raw"],
        {ENCOUNTER_ID: ("count", "unique_transfusions")},
    )(table)
    unique = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "unique_transfusions")}
    )(unique)
    total = get_interface(total).run()
    unique = get_interface(unique).run()
    cohort = cohort.merge(total, on=ENCOUNTER_ID, how="left")
    cohort = cohort.merge(unique, on=ENCOUNTER_ID, how="left")
    cohort["unique_transfusions"] = cohort["unique_transfusions"].fillna(0)
    cohort["total_transfusions"] = cohort["total_transfusions"].fillna(0)

    return cohort


def get_icd_delirium_for_cohort(cohort: pd.DataFrame) -> pd.DataFrame:
    """Get ICD delirium flag based on diagnosis info for cohort.

    Parameters
    ----------
    cohort: pandas.DataFrame
        Cohort data.

    Returns
    -------
    pandas.DataFrame
        Cohort with flag added if ICD code for delirium is present.

    """
    icd_delirium = gemini.diagnoses(include_description=False).query
    icd_delirium = qp.ConditionStartsWith(DIAGNOSIS_CODE, "F05")(icd_delirium)
    icd_delirium = qp.GroupByAggregate(
        [ENCOUNTER_ID, DIAGNOSIS_CODE], {ENCOUNTER_ID: ("count", "icd_delirium")}
    )(icd_delirium)
    icd_delirium = qp.GroupByAggregate(
        [ENCOUNTER_ID, DIAGNOSIS_CODE], {ENCOUNTER_ID: ("count", "icd_delirium")}
    )(icd_delirium)
    icd_delirium = qp.FilterColumns(
        [
            ENCOUNTER_ID,
            DIAGNOSIS_CODE,
            "icd_delirium",
        ]
    )(icd_delirium)
    icd_delirium = get_interface(icd_delirium).run()
    cohort = cohort.merge(icd_delirium, on=ENCOUNTER_ID, how="left")
    cohort["icd_delirium"] = cohort["icd_delirium"].apply(
        lambda x: True if x == 1 else False
    )

    return cohort


def get_diagnoses_for_cohort(cohort: pd.DataFrame) -> pd.DataFrame:
    """Get diagnoses info mapped to CCSR lookup for cohort.

    Parameters
    ----------
    cohort: pandas.DataFrame
        Cohort data.

    Returns
    -------
    pandas.DataFrame
        Cohort with diagnoses info added.

    """
    diagnoses = gemini.diagnoses(include_description=False).query
    ccsr_lookup_table = gemini.get_table(gemini.LOOKUP_CCSR)
    # cohort = cohort.merge(diagnoses, on=ENCOUNTER_ID, how="left")
    for col in ["ccsr_default"]:
        diagnoses = qp.Join(ccsr_lookup_table, on=(col, "ccsr"))(diagnoses)
    diagnoses = gemini.get_interface(diagnoses).run()

    return diagnoses


def get_interventions_for_cohort(cohort: pd.DataFrame) -> pd.DataFrame:
    """Get interventions for cohort.

    Parameters
    ----------
    cohort: pandas.DataFrame
        Cohort data.

    Returns
    -------
    pandas.DataFrame
        Cohort with internventions data added.

    """
    interventions = gemini.interventions().query
    interventions = qp.FilterColumns([ENCOUNTER_ID, "intervention_code"])(interventions)
    interventions = qp.Substring("intervention_code", 0, 3, "intervention_code_substr")(
        interventions
    )
    interventions = gemini.get_interface(interventions).run()
    cohort = cohort.merge(interventions, on=ENCOUNTER_ID, how="left")

    return cohort


def main() -> pd.DataFrame:
    """Run delirium query.

    Returns
    -------
    pandas.DataFrame
        Cohort with internventions data added.

    """
    cohort = get_cohort()
    cohort = get_room_transfers_for_cohort(cohort)
    cohort = get_scu_transfers_for_cohort(cohort)
    cohort = get_er_for_cohort(cohort)
    cohort = get_bt_for_cohort(cohort)
    cohort = get_icd_delirium_for_cohort(cohort)
    cohort = get_diagnoses_for_cohort(cohort)
    cohort = get_interventions_for_cohort(cohort)

    return cohort


cohort = main()

In [ ]:
cohort